### Assignment Problem

A fundamental **combinatorial optimization** problem:

- There are $n$ **tasks** to be completed.
- There are $n$ **workers** available.
- Each worker $i$ has an individual **cost** $c_{ij}$ for performing task $j$.

#### Goal:
Find an assignment of workers to tasks that **minimizes the total cost**.

#### Constraints:
- Each **worker** can be assigned to **at most one** task.
- Each **task** must be assigned to **exactly one** worker.

### Mathematical Formulation
We define the binary decision variables:

$$
x_{ij} = 
\begin{cases}
1, & \text{if worker } i \text{ is assigned to task } j \\
0, & \text{otherwise}
\end{cases}
$$

Let $c_{ij}$ be the cost for worker $i$ to perform task $j$.

#### Objective:

Minimize the total cost:

$$
\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{ij} \cdot x_{ij}
$$

#### Subject to:

Each worker is assigned to at most one task:

$$
\sum_{j=1}^{n} x_{ij} \leq 1 \quad \forall i = 1, \dots, n
$$

Each task is assigned to exactly one worker:

$$
\sum_{i=1}^{n} x_{ij} = 1 \quad \forall j = 1, \dots, n
$$

Binary constraints:

$$
x_{ij} \in \{0, 1\} \quad \forall i, j = 1, \dots, n
$$

### Coding in Python using gurobi

In [49]:
import numpy as np
from gurobipy import GRB, Model

# create input data
cost = np.random.randint(1, 10, (4,4))

In [50]:
# create the model
assignment_model = Model('assignment')

In [51]:
# create the decision variables
x = assignment_model.addVars(cost.shape[0],
                             cost.shape[1],
                             vtype = GRB.BINARY,
                             name = 'x')

In [52]:
# add the constraints
assignment_model.addConstrs((sum(x[i,j] for i in range(cost.shape[0])) <= 1
                           for j in range(cost.shape[1])),
                           name = 'c1')

assignment_model.addConstrs((sum(x[i,j] for j in range(cost.shape[1])) == 1
                            for i in range(cost.shape[0])),
                           name = 'c2')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [53]:
# Set the objective function
obj_fn = sum(cost[i,j]*x[i,j] for i in range(cost.shape[0]) for j in range(cost.shape[1]))
assignment_model.setObjective(obj_fn, GRB.MINIMIZE)

In [61]:
# solve the model
assignment_model.setParam('OutputFlag', 1)
assignment_model.optimize()

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0x6df63a93
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolved: 8 rows, 16 columns, 32 nonzeros

Continuing optimization...


Explored 1 nodes (7 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 13 26 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.300000000000e+01, best bound 1.300000000000e+01, gap 0.0000%
Model Statistics:
Statistics for model 'assignment':
  Problem type        

In [64]:
#statistics of the solution

print('Model Statistics:')
assignment_model.printStats()

Model Statistics:
--------------------
Statistics for model 'assignment':
  Problem type                : MIP
  Linear constraint matrix    : 8 rows, 16 columns, 32 nonzeros
  Variable types              : 0 continuous, 16 integer (16 binary)
  Matrix range                : [1e+00, 1e+00]
  Objective range             : [1e+00, 9e+00]
  Bounds range                : [1e+00, 1e+00]
  RHS range                   : [1e+00, 1e+00]


In [65]:
# outputs of the solution

print('\n\nModel Output:')
if assignment_model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {assignment_model.objVal}")
    for var in assignment_model.getVars():
        print(f"{var.varName} = {var.x}")
else:
    print("Model did not converge to an optimal solution.")



Model Output:
Optimal objective value: 13.0
x[0,0] = 1.0
x[0,1] = -0.0
x[0,2] = -0.0
x[0,3] = -0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 1.0
x[2,0] = -0.0
x[2,1] = -0.0
x[2,2] = 1.0
x[2,3] = -0.0
x[3,0] = -0.0
x[3,1] = 1.0
x[3,2] = -0.0
x[3,3] = -0.0


In [66]:
# show the variables that are used for the optimal solution

print('Objective Function Value: %.2f' % assignment_model.objVal)

# get values of the decision variables
for v in assignment_model.getVars():
    if v.x > 0:
        print(f'{v.varName}: {v.x}')

Objective Function Value: 13.00
x[0,0]: 1.0
x[1,3]: 1.0
x[2,2]: 1.0
x[3,1]: 1.0


### Relaxing the binary constraint

If we relax the binary decision variables to continuous variables, we will obtain the same solution.
This is because the constraints matrix is totally unimodular.<br>
We will check this by doing the following:

In [74]:
assignment_model_2 = Model('assignment_2')

x = assignment_model_2.addVars(cost.shape[0],
                             cost.shape[1],
                             vtype = GRB.CONTINUOUS,
                             lb = 0, 
                             ub = 1,
                             name = 'x')

assignment_model_2.addConstrs((sum(x[i,j] for i in range(cost.shape[0])) <= 1
                           for j in range(cost.shape[1])),
                           name = 'work_load')

assignment_model_2.addConstrs((sum(x[i,j] for j in range(cost.shape[1])) == 1
                            for i in range(cost.shape[0])),
                           name = 'task_completion')

obj_fn = sum(cost[i,j]*x[i,j] for i in range(cost.shape[0]) for j in range(cost.shape[1]))
assignment_model_2.setObjective(obj_fn, GRB.MINIMIZE)


assignment_model_2.setParam('OutputFlag', 1)
assignment_model_2.optimize()

Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 16 columns and 32 nonzeros
Model fingerprint: 0xe44c9c08
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 8 rows, 16 columns, 32 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0000000e+01   1.000000e+00   0.000000e+00      0s
       3    1.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.300000000e+01


In [75]:
print('Model Statistics:')
assignment_model_2.printStats()

Model Statistics:
Statistics for model 'assignment_2':
  Problem type                : LP
  Linear constraint matrix    : 8 rows, 16 columns, 32 nonzeros
  Variable types              : 16 continuous, 0 integer (0 binary)
  Matrix range                : [1e+00, 1e+00]
  Objective range             : [1e+00, 9e+00]
  Bounds range                : [1e+00, 1e+00]
  RHS range                   : [1e+00, 1e+00]


In [76]:
print('\n\nModel Output:')
if assignment_model_2.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {assignment_model.objVal}")
    for var in assignment_model_2.getVars():
        print(f"{var.varName} = {var.x}")
else:
    print("Model did not converge to an optimal solution.")



Model Output:
Optimal objective value: 13.0
x[0,0] = 1.0
x[0,1] = 0.0
x[0,2] = 0.0
x[0,3] = 0.0
x[1,0] = 0.0
x[1,1] = 0.0
x[1,2] = 0.0
x[1,3] = 1.0
x[2,0] = 0.0
x[2,1] = 0.0
x[2,2] = 1.0
x[2,3] = 0.0
x[3,0] = 0.0
x[3,1] = 1.0
x[3,2] = 0.0
x[3,3] = 0.0


In [77]:
print('Objective Function Value: %.2f' % assignment_model.objVal)

# get values of the decision variables
for v in assignment_model_2.getVars():
    if v.x > 0:
        print(f'{v.varName}: {v.x}')

Objective Function Value: 13.00
x[0,0]: 1.0
x[1,3]: 1.0
x[2,2]: 1.0
x[3,1]: 1.0


Remark: LP-Relaxation is valid for linear assignment problem.

In [79]:
assignment_model.printAttr('X')


    Variable            X 
-------------------------
      x[0,0]            1 
      x[1,3]            1 
      x[2,2]            1 
      x[3,1]            1 


In [80]:
assignment_model_2.printAttr('X')


    Variable            X 
-------------------------
      x[0,0]            1 
      x[1,3]            1 
      x[2,2]            1 
      x[3,1]            1 
